# 01.1 Data Extraction - From Source 

This notebook demonstrates how to extract data from the Gamebus-Experiencer application collected using a smartwatch Samsung Active 2. 

<i>References: </i>
<li>Van Gorp, Pieter, and Raoul Nuijten. 2023. “8-Year Evaluation of GameBus: Status Quo in Aiming for an Open Access Platform to Prototype and Test Digital Health Apps.” Proceedings of the ACM on Human-Computer Interaction 7(EICS). <a href="https://doi.org/10.1145/3593223">doi:10.1145/3593223</a>.
<li>Khanshan, Alireza, Pieter Van Gorp, and Panos Markopoulos. 2023. “Experiencer: An Open-Source Context-Sensitive Wearable Experience Sampling Tool: 16th EAI International Conference on Pervasive Computing Technologies for Healthcare, PervasiveHealth 2022” edited by A. Tsanas and A. Triantafyllidis. Pervasive Computing Technologies for Healthcare 315–31. <a href="https://doi.org/10.1007/978-3-031-34586-9_21">doi:10.1007/978-3-031-34586-9_21</a>.



## Setup

In [ ]:
import sys
import pandas as pd
import datetime
sys.path.append('..')

# Import project modules
from config.credentials import AUTHCODE #private
from config.paths import USERS_FILE_PATH #private
from src.extraction.gamebus_client import GameBusClient
from src.extraction.data_collectors import (
    LocationDataCollector, 
    MoodDataCollector,
    ActivityTypeDataCollector,
    HeartRateDataCollector,
    AccelerometerDataCollector,
    NotificationDataCollector
)

# Initialize logging
#from src.utils.logging import setup_logging
#import logging
#logging.basicConfig(level=logging.DEBUG)
#logger = setup_logging(log_level="INFO")
#logger.info("Notebook initialized")

## Initialize GameBus Client

Create a GameBus client for API interactions.

In [6]:
# Initialize the GameBus client
client = GameBusClient(AUTHCODE)
print("GameBus client initialized")

GameBus client initialized


## Load GameBus Users

Load the GameBus users and credentials from a CSV file.

In [7]:
# Load the users
try:
    users_df = pd.read_csv(USERS_FILE_PATH, delimiter=';')
    print(f"Loaded {len(users_df)} users")
except Exception as e:
    # If users file doesn't exist yet, we need to create it
    print(f"Failed to load users file: {e}")

Loaded 2 users


## Select User
Select a user from the list to extract data for.

In [8]:
# Select a user
user_index = 0  # Change this to select a different user
selected_user = users_df.iloc[user_index]
username = selected_user['Username']
password = selected_user['Password']

print(f"Selected user: {username}")

Selected user: info+bm@gamebus.nl


## Authenticate User

Authenticate with the GameBus API and get the player token and ID.

In [9]:
token = client.get_player_token(username, password)
player_id = client.get_player_id(token)

print(f"Player token: {token[:10]}..." if token else "Failed to get token")
print(f"Player ID: {player_id}" if player_id else "Failed to get player ID")

Player token: eyJhbGciOi...
Player ID: 107631


## Define Date Range (Optional)

Define a date range for data extraction. 

In [10]:
# Set to True to enable date filtering
use_date_filter = False

if use_date_filter:
    # Define start date (inclusive)
    start_date = datetime.datetime(2025, 5, 9)  # Format: YYYY, MM, DD
    
    # Define end date (inclusive)
    end_date = datetime.datetime(2025, 6, 9)    # Format: YYYY, MM, DD
    
    print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
else:
    start_date = None
    end_date = None
    print("No date filtering applied - extracting all available data")

No date filtering applied - extracting all available data


## Extract User Data from GameBus

### Extract Accelerometer Data

Extract accelerometer data from GameBus. This includes ts (timestamp), x axis, y axis, and z axis.

In [11]:
# Create an accelerometer data collector
accelerometer_collector = AccelerometerDataCollector(client, token, player_id)

# Collect accelerometer data
accelerometer_data, accelerometer_file = accelerometer_collector.collect(start_date=start_date, end_date=end_date)
#accelerometer_data, accelerometer_file = accelerometer_collector.collect()
print(f"Collected {len(accelerometer_data)} accelerometer data points")
print(f"Data saved to {accelerometer_file}")

# Display a sample of the data
accelerometer_df = pd.DataFrame(accelerometer_data)
if len(accelerometer_df) > 0:
    display(accelerometer_df.head())

Collected 6879729 accelerometer data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_accelerometer.json


,ts,x,y,z,activity_id,activity_date
0,1747751952838,-9.628718,-1.174876,1.287339,3379502,1747752013000
1,1747751952842,-9.664611,-1.036092,0.926022,3379502,1747752013000
2,1747751952864,-9.726824,-0.784846,0.550349,3379502,1747752013000
3,1747751952951,-9.781858,-0.746561,0.538385,3379502,1747752013000
4,1747751952952,-9.690931,-0.873380,0.803988,3379502,1747752013000


### Extract Heart Rate Data

Extract heart rate monitoring data from GameBus. This includes ts (timestamp), hr (heartrate), and pp.

In [12]:
# Create a heart rate data collector
heartrate_collector = HeartRateDataCollector(client, token, player_id)

# Collect heart rate data
heartrate_data, heartrate_file = heartrate_collector.collect(start_date=start_date, end_date=end_date)
#heartrate_data, heartrate_file = heartrate_collector.collect()

print(f"Collected {len(heartrate_data)} heart rate data points")
print(f"Data saved to {heartrate_file}")

# Display a sample of the data
heartrate_df = pd.DataFrame(heartrate_data)
if len(heartrate_df) > 0:
    display(heartrate_df.head())

Collected 1531993 heart rate data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_heartrate.json


,ts,hr,pp,activity_id,activity_date
0,1747751979396,112,0,3379503,1747752013000
1,1747751979476,-10,0,3379503,1747752013000
2,1747751979556,-10,0,3379503,1747752013000
3,1747751979636,-10,0,3379503,1747752013000
4,1747751979716,-10,0,3379503,1747752013000


### Extract Self-Report Data

Extract self-report(mood) logging data from GameBus. This includes valence_state_value, arousal_state_value, stress_state_value, and event_timestamp.

In [13]:
# Create a mood data collector
mood_collector = MoodDataCollector(client, token, player_id)

# Collect mood data
#mood_data, mood_file = mood_collector.collect()
mood_data, mood_file = mood_collector.collect(start_date=start_date, end_date=end_date)
print(f"Collected {len(mood_data)} mood data points")
print(f"Data saved to {mood_file}")

# Display a sample of the data
mood_df = pd.DataFrame(mood_data)
if len(mood_df) > 0:
    display(mood_df.head())

Collected 59 mood data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_mood.json


,VALENCE_STATE_VALUE,AROUSAL_STATE_VALUE,STRESS_STATE_VALUE,EVENT_TIMESTAMP,activity_id,date,gameDescriptor
0,6,4,6,1747580974215,3364013,1747648612000,LOG_MOOD
1,7,3,3,1747570055709,3363688,1747636926000,LOG_MOOD
2,6,4,6,1747548242952,3363008,1747563222000,LOG_MOOD
3,7,7,2,1747490847963,3362352,1747501524000,LOG_MOOD
4,6,6,4,1747407669963,3361078,1747407758000,LOG_MOOD


### Extract Notification Data

Extract notification data from GameBus. This includes action (e.g., received, read) and event_timestamp.

In [14]:
# Create a notification data collector
notification_collector = NotificationDataCollector(client, token, player_id)

# Collect notification data
notification_data, notification_file = notification_collector.collect(start_date=start_date, end_date=end_date)
#notification_data, notification_file = notification_collector.collect()

print(f"Collected {len(notification_data)} notification data points")
print(f"Data saved to {notification_file}")

# Display a sample of the data
notification_df = pd.DataFrame(notification_data)
if len(notification_df) > 0:
    display(notification_df.head())

Collected 175 notification data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_notifications.json


,ACTION,EVENT_TIMESTAMP,activity_id,date,gameDescriptor
0,RECEIVED,1747745108431,3379327,1747745108000,NOTIFICATION(DETAIL)
1,RECEIVED,1747734250029,3378294,1747734250000,NOTIFICATION(DETAIL)
2,RECEIVED,1747723173676,3376830,1747723174000,NOTIFICATION(DETAIL)
3,RECEIVED,1747723174041,3376832,1747723174000,NOTIFICATION(DETAIL)
4,RECEIVED,1747723174247,3376834,1747723174000,NOTIFICATION(DETAIL)


### Extract GPS Location Data

Extract GPS location data from GameBus. This includes latitude, longitude, altitude, speed, error, timestamp, and arm.

In [15]:
# Create a location data collector
location_collector = LocationDataCollector(client, token, player_id)

# Collect location data
location_data, location_file = location_collector.collect(start_date=start_date, end_date=end_date)
#location_data, location_file = location_collector.collect()
print(f"Collected {len(location_data)} location data points")
print(f"Data saved to {location_file}")

# Display a sample of the data
location_df = pd.DataFrame(location_data)
if len(location_df) > 0:
    display(location_df.head())

Collected 18974 location data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_location.json


,LATITUDE,LONGITUDE,ALTIDUDE,SPEED,ERROR,TIMESTAMP,ARM,activity_id,date,gameDescriptor
0,200,200,0,0,0,1747752369,Arm 2,3379510,1747752429000,GEOFENCE
1,200,200,0,0,0,1747752309,Arm 2,3379509,1747752369000,GEOFENCE
2,200,200,0,0,0,1747752248,Arm 2,3379508,1747752309000,GEOFENCE
3,200,200,0,0,0,1747752188,Arm 2,3379507,1747752248000,GEOFENCE
4,200,200,0,0,0,1747752128,Arm 2,3379506,1747752188000,GEOFENCE


### Extract Activity Type Data

Extract activity type data from GameBus. This includes src, ts (timestamp), type, speed, steps, walks, runs, freq, distance, and cals.

In [16]:
# Create an activity type data collector
activity_collector = ActivityTypeDataCollector(client, token, player_id)

# Collect activity type data
activity_data, activity_file = activity_collector.collect(start_date=start_date, end_date=end_date)
#activity_data, activity_file = activity_collector.collect()

print(f"Collected {len(activity_data)} activity data points")
print(f"Data saved to {activity_file}")

# Display a sample of the data
activity_df = pd.DataFrame(activity_data)
if len(activity_df) > 0:
    display(activity_df.head())

Collected 3526 activity data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_activity_type.json


,src,ts,type,speed,steps,walks,runs,freq,distance,cals,activity_id,activity_date
0,p,1747751892337,NOT_MOVING,0.0,4005.0,3889.0,3.0,0.0,3033.040039,140.429993,3379499,1747751892000
1,p,1747751701311,NOT_MOVING,0.0,4005.0,3889.0,3.0,0.0,3033.040039,140.429993,3379489,1747751701000
2,p,1747751582022,NOT_MOVING,0.0,4005.0,3889.0,3.0,0.0,3033.040039,140.429993,3379486,1747751582000
3,p,1747750491551,NOT_MOVING,0.0,3938.0,3822.0,3.0,0.0,2982.760010,138.179993,3379464,1747750492000
4,p,1747749623942,NOT_MOVING,0.0,3938.0,3822.0,3.0,0.0,2982.760010,138.179993,3379446,1747749624000


## Data Summary

In [17]:
# Create a summary of collected data
summary = {
    "Location": len(location_data),
    "Mood": len(mood_data),
    "Activity": len(activity_data),
    "Heart Rate": len(heartrate_data),
    "Accelerometer": len(accelerometer_data),
    "Notification": len(notification_data)
}

summary_df = pd.DataFrame({
    "Data Type": list(summary.keys()),
    "Count": list(summary.values())
})

# Display summary
display(summary_df)

print(f"\nTotal data points collected: {sum(summary.values())}")

,Data Type,Count
0,Location,18974
1,Mood,59
2,Activity,3526
3,Heart Rate,1531993
4,Accelerometer,6879729
5,Notification,175



Total data points collected: 8434456
